# Task description
- Classify the speakers of given features.
- Main goal: Learn how to use transformer.
- Baselines:
  - Easy: Run sample code and know how to use transformer.
  - Medium: Know how to adjust parameters of transformer.
  - Strong: Construct [conformer](https://arxiv.org/abs/2005.08100) which is a variety of transformer. 
  - Boss: Implement [Self-Attention Pooling](https://arxiv.org/pdf/2008.01077v1.pdf) & [Additive Margin Softmax](https://arxiv.org/pdf/1801.05599.pdf) to further boost the performance.

- Other links
  - Kaggle: [link](https://www.kaggle.com/t/ac77388c90204a4c8daebeddd40ff916)
  - Slide: [link](https://docs.google.com/presentation/d/1HLAj7UUIjZOycDe7DaVLSwJfXVd3bXPOyzSb6Zk3hYU/edit?usp=sharing)
  - Data: [link](https://drive.google.com/drive/folders/1vI1kuLB-q1VilIftiwnPOCAeOOFfBZge?usp=sharing)

# Download dataset
- Data is [here](https://drive.google.com/drive/folders/1vI1kuLB-q1VilIftiwnPOCAeOOFfBZge?usp=sharing)

## Fix Random Seed

In [1]:
import numpy as np
import torch
import random

def set_seed(seed):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

set_seed(87)

# Data

## Dataset
- Original dataset is [Voxceleb2](https://www.robots.ox.ac.uk/~vgg/data/voxceleb/vox2.html).
- The [license](https://creativecommons.org/licenses/by/4.0/) and [complete version](https://www.robots.ox.ac.uk/~vgg/data/voxceleb/files/license.txt) of Voxceleb2.
- We randomly select 600 speakers from Voxceleb2.
- Then preprocess the raw waveforms into mel-spectrograms.

- Args:
  - data_dir: The path to the data directory.
  - metadata_path: The path to the metadata.
  - segment_len: The length of audio segment for training. 
- The architecture of data directory \\
  - data directory \\
  |---- metadata.json \\
  |---- testdata.json \\
  |---- mapping.json \\
  |---- uttr-{random string}.pt \\

- The information in metadata
  - "n_mels": The dimention of mel-spectrogram.
  - "speakers": A dictionary. 
    - Key: speaker ids.
    - value: "feature_path" and "mel_len"


For efficiency, we segment the mel-spectrograms into segments in the traing step.

In [2]:
!pip install conformer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.7 MB/s eta 0:00:00


In [3]:
import os
import json
import torch
import random
from pathlib import Path
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence
from conformer import ConformerBlock
 
class myDataset(Dataset):
    def __init__(self, data_dir, segment_len=128):
        self.data_dir = data_dir
        self.segment_len = segment_len

        # Load the mapping from speaker neme to their corresponding id. 
        mapping_path = Path(data_dir) / "mapping.json"
        mapping = json.load(mapping_path.open())
        self.speaker2id = mapping["speaker2id"]

        # Load metadata of training data.
        metadata_path = Path(data_dir) / "metadata.json"
        metadata = json.load(open(metadata_path))["speakers"]

        # Get the total number of speaker.
        self.speaker_num = len(metadata.keys())
        self.data = []
        for speaker in metadata.keys():
            for utterances in metadata[speaker]:
                self.data.append([utterances["feature_path"], self.speaker2id[speaker]])

    def __len__(self):
            return len(self.data)

    def __getitem__(self, index):
        feat_path, speaker = self.data[index]
        # Load preprocessed mel-spectrogram.
        mel = torch.load(os.path.join(self.data_dir, feat_path))

        # Segmemt mel-spectrogram into "segment_len" frames.
        if len(mel) > self.segment_len:
            # Randomly get the starting point of the segment.
            start = random.randint(0, len(mel) - self.segment_len)
            # Get a segment with "segment_len" frames.
            mel = torch.FloatTensor(mel[start:start+self.segment_len])
        else:
            mel = torch.FloatTensor(mel)
        # Turn the speaker id into long for computing loss later.
        speaker = torch.FloatTensor([speaker]).long()
        return mel, speaker
 
    def get_speaker_number(self):
        return self.speaker_num

## Dataloader
- Split dataset into training dataset(90%) and validation dataset(10%).
- Create dataloader to iterate the data.

In [4]:
import torch
from torch.utils.data import DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence


def collate_batch(batch):
    # Process features within a batch.
    """Collate a batch of data."""
    mel, speaker = zip(*batch)
    # Because we train the model batch by batch, we need to pad the features in the same batch to make their lengths the same.
    mel = pad_sequence(mel, batch_first=True, padding_value=-20)    # pad log 10^(-20) which is very small value.
    # mel: (batch size, length, 40)
    return mel, torch.FloatTensor(speaker).long()


def get_dataloader(data_dir, batch_size, n_workers):
    """Generate dataloader"""
    dataset = myDataset(data_dir)
    speaker_num = dataset.get_speaker_number()
    # Split dataset into training dataset and validation dataset
    trainlen = int(0.9 * len(dataset))
    lengths = [trainlen, len(dataset) - trainlen]
    trainset, validset = random_split(dataset, lengths)

    train_loader = DataLoader(
        trainset,
        batch_size=batch_size,
        shuffle=True,
        drop_last=True,
        num_workers=n_workers,
        pin_memory=True,
        collate_fn=collate_batch,
    )
    valid_loader = DataLoader(
        validset,
        batch_size=batch_size,
        num_workers=n_workers,
        drop_last=True,
        pin_memory=True,
        collate_fn=collate_batch,
    )

    return train_loader, valid_loader, speaker_num

# Model
- TransformerEncoderLayer:
  - Base transformer encoder layer in [Attention Is All You Need](https://arxiv.org/abs/1706.03762)
  - Parameters:
    - d_model: the number of expected features of the input (required).

    - nhead: the number of heads of the multiheadattention models (required).

    - dim_feedforward: the dimension of the feedforward network model (default=2048).

    - dropout: the dropout value (default=0.1).

    - activation: the activation function of intermediate layer, relu or gelu (default=relu).

- TransformerEncoder:
  - TransformerEncoder is a stack of N transformer encoder layers
  - Parameters:
    - encoder_layer: an instance of the TransformerEncoderLayer() class (required).

    - num_layers: the number of sub-encoder-layers in the encoder (required).

    - norm: the layer normalization component (optional).

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.nn import CrossEntropyLoss

In [6]:
class SelfAttentionPooling(nn.Module):
    def __init__(self, input_dim):
        super(SelfAttentionPooling, self).__init__()
        self.W = nn.Linear(input_dim, 1)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, batch_rep):
        """
        input:
            batch_rep : size (N, T, H), N: batch size, T: sequence length, H: Hidden dimension
      
        attention_weight:
            att_w : size (N, T, 1)
    
        return:
            utter_rep: size (N, H)
        """
        out = self.W(batch_rep).squeeze(-1)
        att_w = self.softmax(out).unsqueeze(-1)
        utter_rep = torch.sum(batch_rep * att_w, dim=1)

        return utter_rep

In [7]:
# Transformer Model
class TransformerClassifier(nn.Module):
    def __init__(self, d_model=80, n_spks=600, dropout=0.1):
        super().__init__()
        # Project the dimension of features from that of input into d_model.
        self.prenet = nn.Linear(40, d_model)
        self.encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, dim_feedforward=256, nhead=2
        )
        # self.encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=2)

        # Project the the dimension of features from d_model into speaker nums.
        self.pred_layer = nn.Sequential(
            nn.Linear(d_model, d_model),
            nn.ReLU(),
            nn.Linear(d_model, n_spks),
        )

    def forward(self, mels):
        """
        args:
            mels: (batch size, length, 40)
        return:
            out: (batch size, n_spks)
        """
        out = self.prenet(mels)
        out = out.permute(1, 0, 2)
        out = self.encoder_layer(out)
        out = out.transpose(0, 1)
        stats = out.mean(dim=1)

        out = self.pred_layer(stats)
        return out

In [8]:
# Transformer Model
class TransformerClassifierPooling(nn.Module):
    def __init__(self, d_model=80, n_spks=600, dropout=0.1):
        super().__init__()
        # Project the dimension of features from that of input into d_model.
        self.prenet = nn.Linear(40, d_model)
        self.encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, dim_feedforward=256, nhead=2
        )
        # self.encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=2)

        # Project the the dimension of features from d_model into speaker nums.

        self.attention_pooling = SelfAttentionPooling(d_model)
        self.linear_model = nn.Linear(d_model, n_spks)

    def forward(self, mels):
        """
        args:
            mels: (batch size, length, 40)
        return:
            out: (batch size, n_spks)
        """
        out = self.prenet(mels)
        out = out.permute(1, 0, 2)
        out = self.encoder_layer(out)
        out = out.transpose(0, 1)

        out = self.attention_pooling(out)
        out = self.linear_model(out)
        return out

In [9]:
# Confomer Model 
class ConformerClassifier(nn.Module):
    def __init__(self, d_model=80, n_spks=600, dropout=0.1):
        super().__init__()
        # Project the dimension of features from that of input into d_model.
        self.prenet = nn.Linear(40, d_model)
        
        self.block1 = ConformerBlock(
                    dim = d_model,
                    dim_head = 32,
                    heads = 8,
                    ff_mult = 4,
                    conv_expansion_factor = 2,
                    conv_kernel_size = 31,
                    attn_dropout = 0.,
                    ff_dropout = 0.,
                    conv_dropout = 0.
                    )
        
        self.block2 = ConformerBlock(
                    dim = d_model,
                    dim_head = 32,
                    heads = 8,
                    ff_mult = 4,
                    conv_expansion_factor = 2,
                    conv_kernel_size = 31,
                    attn_dropout = 0.,
                    ff_dropout = 0.,
                    conv_dropout = 0.
                    )
        
        self.block3 = ConformerBlock(
                    dim = d_model,
                    dim_head = 32,
                    heads = 8,
                    ff_mult = 4,
                    conv_expansion_factor = 2,
                    conv_kernel_size = 31,
                    attn_dropout = 0.1,
                    ff_dropout = 0.1,
                    conv_dropout = 0.1
                    )
        
        # Project the the dimension of features from d_model into speaker nums.
        self.pred_layer = nn.Sequential(
            nn.Linear(d_model, d_model),
            nn.ReLU(),
            nn.Linear(d_model, n_spks),
        )

    def forward(self, mels):
        """
        args:
            mels: (batch size, length, 40)
        return:
            out: (batch size, n_spks)
        """
        # out: (batch size, length, d_model)
        out = self.prenet(mels)
        # The encoder layer expect features in the shape of (length, batch size, d_model).
        out = self.block1(out)
        out = self.block2(out)
        out = self.block3(out)
        # out: (batch, n_spks)
        out = out.mean(dim=1)
        out = self.pred_layer(out)
        return out

In [10]:
# Confomer Model 
class ConformerClassifierPooling(nn.Module):
    def __init__(self, d_model=80, n_spks=600, dropout=0.1):
        super().__init__()
        # Project the dimension of features from that of input into d_model.
        self.prenet = nn.Linear(40, d_model)
        
        self.block1 = ConformerBlock(
                    dim = d_model,
                    dim_head = 32,
                    heads = 8,
                    ff_mult = 4,
                    conv_expansion_factor = 2,
                    conv_kernel_size = 31,
                    attn_dropout = 0.,
                    ff_dropout = 0.,
                    conv_dropout = 0.
                    )
        
        self.block2 = ConformerBlock(
                    dim = d_model,
                    dim_head = 32,
                    heads = 8,
                    ff_mult = 4,
                    conv_expansion_factor = 2,
                    conv_kernel_size = 31,
                    attn_dropout = 0.,
                    ff_dropout = 0.,
                    conv_dropout = 0.
                    )
        
        self.block3 = ConformerBlock(
                    dim = d_model,
                    dim_head = 32,
                    heads = 8,
                    ff_mult = 4,
                    conv_expansion_factor = 2,
                    conv_kernel_size = 31,
                    attn_dropout = 0.1,
                    ff_dropout = 0.1,
                    conv_dropout = 0.1
                    )
        
        # Project the the dimension of features from d_model into speaker nums.
        self.attention_pooling = SelfAttentionPooling(d_model)
        self.linear_model = nn.Linear(d_model, n_spks)

    def forward(self, mels):
        """
        args:
            mels: (batch size, length, 40)
        return:
            out: (batch size, n_spks)
        """
        # out: (batch size, length, d_model)
        out = self.prenet(mels)
        # The encoder layer expect features in the shape of (length, batch size, d_model).
        out = self.block1(out)
        out = self.block2(out)
        out = self.block3(out)
        # out: (batch, n_spks)
        out = self.attention_pooling(out)
        out = self.linear_model(out)
        return out

# Learning rate schedule
- For transformer architecture, the design of learning rate schedule is different from that of CNN.
- Previous works show that the warmup of learning rate is useful for training models with transformer architectures.
- The warmup schedule
  - Set learning rate to 0 in the beginning.
  - The learning rate increases linearly from 0 to initial learning rate during warmup period.

In [11]:
import math

import torch
from torch.optim import Optimizer
from torch.optim.lr_scheduler import LambdaLR


def get_cosine_schedule_with_warmup(
    optimizer: Optimizer,
    num_warmup_steps: int,
    num_training_steps: int,
    num_cycles: float = 0.5,
    last_epoch: int = -1,
    ):
    """
    Create a schedule with a learning rate that decreases following the values of the cosine function between the
    initial lr set in the optimizer to 0, after a warmup period during which it increases linearly between 0 and the
    initial lr set in the optimizer.

    Args:
        optimizer (:class:`~torch.optim.Optimizer`):
        The optimizer for which to schedule the learning rate.
        num_warmup_steps (:obj:`int`):
        The number of steps for the warmup phase.
        num_training_steps (:obj:`int`):
        The total number of training steps.
        num_cycles (:obj:`float`, `optional`, defaults to 0.5):
        The number of waves in the cosine schedule (the defaults is to just decrease from the max value to 0
        following a half-cosine).
        last_epoch (:obj:`int`, `optional`, defaults to -1):
        The index of the last epoch when resuming training.

    Return:
        :obj:`torch.optim.lr_scheduler.LambdaLR` with the appropriate schedule.
    """
    def lr_lambda(current_step):
        # Warmup
        if current_step < num_warmup_steps:
            return float(current_step) / float(max(1, num_warmup_steps))
        # decadence
        progress = float(current_step - num_warmup_steps) / float(
            max(1, num_training_steps - num_warmup_steps)
        )
        return max(
            0.0, 0.5 * (1.0 + math.cos(math.pi * float(num_cycles) * 2.0 * progress))
        )

    return LambdaLR(optimizer, lr_lambda, last_epoch)

# Model Function
- Model forward function.

In [12]:
import torch


def model_fn(batch, model, criterion, device, isAM = False):
    """Forward a batch through the model."""

    mels, labels = batch
    mels = mels.to(device)
    labels = labels.to(device)

    outs = model(mels)
    if isAM:
        loss = criterion(outs, labels, scale=30.0, margin=0.4)
        # Get the speaker id with highest probability.
        preds = outs.argmax(1)
        # Compute accuracy.
        accuracy = torch.mean((preds == labels).float())
        
        return loss, accuracy
    
    loss = criterion(outs, labels)
    # Get the speaker id with highest probability.
    preds = outs.argmax(1)
    # Compute accuracy.
    accuracy = torch.mean((preds == labels).float())

    return loss, accuracy

# Validate
- Calculate accuracy of the validation set.

In [13]:
from tqdm import tqdm
import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import DataLoader, random_split


def valid(dataloader, model, criterion, device, isAM = False): 
    """Validate on validation set."""

    model.eval()
    running_loss = 0.0
    running_accuracy = 0.0
    pbar = tqdm(total=len(dataloader.dataset), ncols=0, desc="Valid", unit=" uttr")
    
    for i, batch in enumerate(dataloader):
        with torch.no_grad():
            loss, accuracy = model_fn(batch, model, criterion, device, isAM)
            running_loss += loss.item()
            running_accuracy += accuracy.item()

        pbar.update(dataloader.batch_size)
        pbar.set_postfix(
            loss=f"{running_loss / (i+1):.2f}",
            accuracy=f"{running_accuracy / (i+1):.2f}",
        )

    pbar.close()
    model.train()

    return running_accuracy / len(dataloader)

In [14]:
class AMSoftmax(nn.Module):
    def __init__(self):
        super(AMSoftmax, self).__init__()

    def forward(self, input, target, scale=10.0, margin=0.35):
        # self.it += 1
        cos_theta = input
        target = target.view(-1, 1)  # size=(B,1)

        index = cos_theta.data * 0.0  # size=(B,Classnum)
        index.scatter_(1, target.data.view(-1, 1), 1)
        index = index.byte()
        index = Variable(index)

        output = cos_theta * 1.0  # size=(B,Classnum)
        output[index] -= margin
        output = output * scale

        logpt = F.log_softmax(output)
        logpt = logpt.gather(1, target)
        logpt = logpt.view(-1)

        loss = -1 * logpt
        loss = loss.mean()

        return loss


# Transformer Model Train

In [15]:
def parse_args():
    """arguments"""
    config = {
        "data_dir": "/kaggle/input/cs4650dataset/Dataset",
        "save_path": "model.ckpt",
        "batch_size": 32,
        "n_workers": 8,
        "valid_steps": 2000,
        "warmup_steps": 1000,
        "save_steps": 10000,
        "total_steps": 100000,
    }

    return config


def main(
    data_dir,
    save_path,
    batch_size,
    n_workers,
    valid_steps,
    warmup_steps,
    total_steps,
    save_steps,
):
    """Main function."""
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"[Info]: Use {device} now!")

    train_loader, valid_loader, speaker_num = get_dataloader(data_dir, batch_size, n_workers)
    train_iterator = iter(train_loader)
    print(f"[Info]: Finish loading data!",flush = True)

    transformer_model = TransformerClassifier(n_spks=speaker_num).to(device)
    criterion = CrossEntropyLoss()
    optimizer = AdamW(transformer_model.parameters(), lr=1e-3)
    scheduler = get_cosine_schedule_with_warmup(optimizer, warmup_steps, total_steps)
    print(f"[Info]: Finish creating model!",flush = True)

    best_accuracy = -1.0
    best_state_dict = None

    pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

    for step in range(total_steps):
        # Get data
        try:
            batch = next(train_iterator)
        except StopIteration:
            train_iterator = iter(train_loader)
            batch = next(train_iterator)

        loss, accuracy = model_fn(batch, transformer_model, criterion, device)
        batch_loss = loss.item()
        batch_accuracy = accuracy.item()

        # Updata transformer model
        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

        # Log
        pbar.update()
        pbar.set_postfix(
            loss=f"{batch_loss:.2f}",
            accuracy=f"{batch_accuracy:.2f}",
            step=step + 1,
        )

        # Do validation
        if (step + 1) % valid_steps == 0:
            pbar.close()

            valid_accuracy = valid(valid_loader, transformer_model, criterion, device)

            # keep the best model
            if valid_accuracy > best_accuracy:
                best_accuracy = valid_accuracy
                best_state_dict = transformer_model.state_dict()

            pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

        # Save the best model so far.
        if (step + 1) % save_steps == 0 and best_state_dict is not None:
            torch.save(best_state_dict, save_path)
            pbar.write(f"Step {step + 1}, best model saved. (accuracy={best_accuracy:.4f})")

    pbar.close()


if __name__ == "__main__":
    main(**parse_args())

[Info]: Use cuda now!


/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


[Info]: Finish loading data!
[Info]: Finish creating model!


Train: 100% 2000/2000 [01:27<00:00, 22.90 step/s, accuracy=0.09, loss=3.96, step=2000]
Valid: 100% 5664/5667 [00:08<00:00, 684.70 uttr/s, accuracy=0.16, loss=4.09]
Train: 100% 2000/2000 [00:43<00:00, 46.42 step/s, accuracy=0.16, loss=3.70, step=4000]
Valid: 100% 5664/5667 [00:03<00:00, 1884.13 uttr/s, accuracy=0.25, loss=3.52]
Train: 100% 2000/2000 [00:40<00:00, 48.96 step/s, accuracy=0.41, loss=2.75, step=6000]
Valid: 100% 5664/5667 [00:03<00:00, 1833.39 uttr/s, accuracy=0.35, loss=2.99]
Train: 100% 2000/2000 [00:41<00:00, 48.56 step/s, accuracy=0.41, loss=2.65, step=8000]
Valid: 100% 5664/5667 [00:03<00:00, 1677.82 uttr/s, accuracy=0.38, loss=2.79]
Train: 100% 2000/2000 [00:41<00:00, 47.94 step/s, accuracy=0.28, loss=3.15, step=1e+4]
Valid: 100% 5664/5667 [00:03<00:00, 1773.72 uttr/s, accuracy=0.42, loss=2.62]
Train:   0% 10/2000 [00:00<00:41, 48.46 step/s, accuracy=0.47, loss=2.52, step=1e+4]

Step 10000, best model saved. (accuracy=0.4160)


...

Step 80000, best model saved. (accuracy=0.7067)


Train: 100% 2000/2000 [00:40<00:00, 49.48 step/s, accuracy=0.78, loss=0.65, step=82000]
Valid: 100% 5664/5667 [00:03<00:00, 1779.94 uttr/s, accuracy=0.70, loss=1.34]
Train: 100% 2000/2000 [00:41<00:00, 47.96 step/s, accuracy=0.78, loss=0.96, step=84000]
Valid: 100% 5664/5667 [00:03<00:00, 1776.16 uttr/s, accuracy=0.70, loss=1.33]
Train: 100% 2000/2000 [00:41<00:00, 47.72 step/s, accuracy=0.59, loss=1.00, step=86000]
Valid: 100% 5664/5667 [00:03<00:00, 1696.91 uttr/s, accuracy=0.71, loss=1.30]
Train: 100% 2000/2000 [00:41<00:00, 48.41 step/s, accuracy=0.81, loss=1.05, step=88000]
Valid: 100% 5664/5667 [00:03<00:00, 1838.36 uttr/s, accuracy=0.70, loss=1.32]
Train: 100% 2000/2000 [00:41<00:00, 47.64 step/s, accuracy=0.81, loss=0.54, step=9e+4] 
Valid: 100% 5664/5667 [00:03<00:00, 1811.23 uttr/s, accuracy=0.71, loss=1.29]
Train:   1% 11/2000 [00:00<00:36, 53.91 step/s, accuracy=0.78, loss=0.71, step=9e+4]

Step 90000, best model saved. (accuracy=0.7092)


Train: 100% 2000/2000 [00:40<00:00, 49.08 step/s, accuracy=0.75, loss=1.26, step=92000]
Valid: 100% 5664/5667 [00:03<00:00, 1807.83 uttr/s, accuracy=0.70, loss=1.29]
Train: 100% 2000/2000 [00:41<00:00, 47.65 step/s, accuracy=0.69, loss=0.99, step=94000]
Valid: 100% 5664/5667 [00:03<00:00, 1842.27 uttr/s, accuracy=0.71, loss=1.31]
Train: 100% 2000/2000 [00:44<00:00, 45.40 step/s, accuracy=0.84, loss=0.55, step=96000]
Valid: 100% 5664/5667 [00:03<00:00, 1689.23 uttr/s, accuracy=0.70, loss=1.33]
Train: 100% 2000/2000 [00:41<00:00, 47.99 step/s, accuracy=0.75, loss=1.00, step=98000]
Valid: 100% 5664/5667 [00:03<00:00, 1561.12 uttr/s, accuracy=0.70, loss=1.31]
Train: 100% 2000/2000 [00:42<00:00, 47.32 step/s, accuracy=0.78, loss=0.95, step=1e+5] 
Valid: 100% 5664/5667 [00:03<00:00, 1632.75 uttr/s, accuracy=0.71, loss=1.26]
Train:   0% 0/2000 [00:00<?, ? step/s]


Step 100000, best model saved. (accuracy=0.7135)


In [16]:
def parse_args():
    """arguments"""
    config = {
        "data_dir": "/kaggle/input/cs4650dataset/Dataset",
        "save_path": "model.ckpt",
        "batch_size": 32,
        "n_workers": 8,
        "valid_steps": 2000,
        "warmup_steps": 1000,
        "save_steps": 10000,
        "total_steps": 100000,
    }

    return config


def main(
    data_dir,
    save_path,
    batch_size,
    n_workers,
    valid_steps,
    warmup_steps,
    total_steps,
    save_steps,
):
    """Main function."""
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"[Info]: Use {device} now!")

    train_loader, valid_loader, speaker_num = get_dataloader(data_dir, batch_size, n_workers)
    train_iterator = iter(train_loader)
    print(f"[Info]: Finish loading data!",flush = True)

    transformer_model = TransformerClassifier(n_spks=speaker_num).to(device)
    criterion = AMSoftmax()
    optimizer = AdamW(transformer_model.parameters(), lr=1e-3)
    scheduler = get_cosine_schedule_with_warmup(optimizer, warmup_steps, total_steps)
    print(f"[Info]: Finish creating model!",flush = True)

    best_accuracy = -1.0
    best_state_dict = None

    pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

    for step in range(total_steps):
        # Get data
        try:
            batch = next(train_iterator)
        except StopIteration:
            train_iterator = iter(train_loader)
            batch = next(train_iterator)

        loss, accuracy = model_fn(batch, transformer_model, criterion, device, True)
        batch_loss = loss.item()
        batch_accuracy = accuracy.item()

        # Updata transformer model
        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

        # Log
        pbar.update()
        pbar.set_postfix(
            loss=f"{batch_loss:.2f}",
            accuracy=f"{batch_accuracy:.2f}",
            step=step + 1,
        )

        # Do validation
        if (step + 1) % valid_steps == 0:
            pbar.close()

            valid_accuracy = valid(valid_loader, transformer_model, criterion, device)

            # keep the best model
            if valid_accuracy > best_accuracy:
                best_accuracy = valid_accuracy
                best_state_dict = transformer_model.state_dict()

            pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

        # Save the best model so far.
        if (step + 1) % save_steps == 0 and best_state_dict is not None:
            torch.save(best_state_dict, save_path)
            pbar.write(f"Step {step + 1}, best model saved. (accuracy={best_accuracy:.4f})")

    pbar.close()


if __name__ == "__main__":
    main(**parse_args())

[Info]: Use cuda now!
[Info]: Finish loading data!
[Info]: Finish creating model!


Train:   0% 0/2000 [00:00<?, ? step/s]/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:16: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at /usr/local/src/pytorch/aten/src/ATen/native/IndexingUtils.h:27.)
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
/opt/conda/lib/python3.7/site-packages/torch/autograd/__init__.py:199: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at /usr/local/src/pytorch/aten/src/ATen/native/IndexingUtils.h:27.)
  allow_unreachable=True, accumulate_grad=True)  # Calls into the C++ engine to run the backward pass
Train: 100% 2000/2000 [00:42<00:00, 46.61 step/s, accuracy=0.03, loss=17.11, step=2000]
Valid: 100% 5664/5667 [00:03<00:00, 1822.04 uttr/s, accur

Step 10000, best model saved. (accuracy=0.2426)


Train: 100% 2000/2000 [00:44<00:00, 44.46 step/s, accuracy=0.22, loss=15.59, step=12000]
Valid: 100% 5664/5667 [00:03<00:00, 1737.86 uttr/s, accuracy=0.27, loss=7.33]
Train: 100% 2000/2000 [00:46<00:00, 43.20 step/s, accuracy=0.31, loss=14.85, step=14000]
Valid: 100% 5664/5667 [00:03<00:00, 1619.81 uttr/s, accuracy=0.30, loss=7.10]
Train: 100% 2000/2000 [00:44<00:00, 45.22 step/s, accuracy=0.34, loss=13.92, step=16000]
Valid: 100% 5664/5667 [00:03<00:00, 1772.92 uttr/s, accuracy=0.36, loss=6.71]
Train: 100% 2000/2000 [00:43<00:00, 46.37 step/s, accuracy=0.50, loss=12.60, step=18000]
Valid: 100% 5664/5667 [00:03<00:00, 1759.22 uttr/s, accuracy=0.39, loss=6.49]
Train: 100% 2000/2000 [00:43<00:00, 46.13 step/s, accuracy=0.47, loss=12.66, step=2e+4] 
Valid: 100% 5664/5667 [00:03<00:00, 1765.30 uttr/s, accuracy=0.41, loss=6.16]
Train:   0% 9/2000 [00:00<00:36, 54.07 step/s, accuracy=0.47, loss=12.73, step=2e+4]

Step 20000, best model saved. (accuracy=0.4107)


...

Step 80000, best model saved. (accuracy=0.6838)


Train: 100% 2000/2000 [00:42<00:00, 46.87 step/s, accuracy=0.69, loss=8.98, step=82000] 
Valid: 100% 5664/5667 [00:03<00:00, 1812.85 uttr/s, accuracy=0.68, loss=3.29]
Train: 100% 2000/2000 [00:43<00:00, 46.49 step/s, accuracy=0.72, loss=8.77, step=84000] 
Valid: 100% 5664/5667 [00:03<00:00, 1787.25 uttr/s, accuracy=0.69, loss=3.27]
Train: 100% 2000/2000 [00:42<00:00, 47.04 step/s, accuracy=0.69, loss=7.20, step=86000] 
Valid: 100% 5664/5667 [00:03<00:00, 1779.66 uttr/s, accuracy=0.69, loss=3.25]
Train: 100% 2000/2000 [00:44<00:00, 45.44 step/s, accuracy=0.84, loss=6.49, step=88000] 
Valid: 100% 5664/5667 [00:03<00:00, 1775.37 uttr/s, accuracy=0.69, loss=3.25]
Train: 100% 2000/2000 [00:55<00:00, 35.76 step/s, accuracy=0.75, loss=7.74, step=9e+4]  
Valid: 100% 5664/5667 [00:03<00:00, 1797.73 uttr/s, accuracy=0.69, loss=3.29]
Train:   0% 7/2000 [00:00<01:10, 28.19 step/s, accuracy=0.78, loss=5.57, step=9e+4]

Step 90000, best model saved. (accuracy=0.6944)


Train: 100% 2000/2000 [00:49<00:00, 40.32 step/s, accuracy=0.84, loss=5.58, step=92000] 
Valid: 100% 5664/5667 [00:03<00:00, 1802.34 uttr/s, accuracy=0.68, loss=3.27]
Train: 100% 2000/2000 [00:53<00:00, 37.14 step/s, accuracy=0.84, loss=6.65, step=94000] 
Valid: 100% 5664/5667 [00:03<00:00, 1785.38 uttr/s, accuracy=0.70, loss=3.20]
Train: 100% 2000/2000 [00:48<00:00, 41.33 step/s, accuracy=0.72, loss=6.09, step=96000] 
Valid: 100% 5664/5667 [00:03<00:00, 1528.41 uttr/s, accuracy=0.69, loss=3.23]
Train: 100% 2000/2000 [00:52<00:00, 37.82 step/s, accuracy=0.72, loss=7.20, step=98000] 
Valid: 100% 5664/5667 [00:03<00:00, 1811.92 uttr/s, accuracy=0.70, loss=3.22]
Train: 100% 2000/2000 [00:51<00:00, 38.92 step/s, accuracy=0.78, loss=5.90, step=1e+5]  
Valid: 100% 5664/5667 [00:03<00:00, 1773.10 uttr/s, accuracy=0.69, loss=3.25]
Train:   0% 0/2000 [00:00<?, ? step/s]


Step 100000, best model saved. (accuracy=0.7006)


In [17]:
def parse_args():
    """arguments"""
    config = {
        "data_dir": "/kaggle/input/cs4650dataset/Dataset",
        "save_path": "model.ckpt",
        "batch_size": 32,
        "n_workers": 8,
        "valid_steps": 2000,
        "warmup_steps": 1000,
        "save_steps": 10000,
        "total_steps": 100000,
    }

    return config


def main(
    data_dir,
    save_path,
    batch_size,
    n_workers,
    valid_steps,
    warmup_steps,
    total_steps,
    save_steps,
):
    """Main function."""
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"[Info]: Use {device} now!")

    train_loader, valid_loader, speaker_num = get_dataloader(data_dir, batch_size, n_workers)
    train_iterator = iter(train_loader)
    print(f"[Info]: Finish loading data!",flush = True)

    transformer_model = TransformerClassifierPooling(n_spks=speaker_num).to(device)
    criterion = CrossEntropyLoss()
    optimizer = AdamW(transformer_model.parameters(), lr=1e-3)
    scheduler = get_cosine_schedule_with_warmup(optimizer, warmup_steps, total_steps)
    print(f"[Info]: Finish creating model!",flush = True)

    best_accuracy = -1.0
    best_state_dict = None

    pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

    for step in range(total_steps):
        # Get data
        try:
            batch = next(train_iterator)
        except StopIteration:
            train_iterator = iter(train_loader)
            batch = next(train_iterator)

        loss, accuracy = model_fn(batch, transformer_model, criterion, device)
        batch_loss = loss.item()
        batch_accuracy = accuracy.item()

        # Updata transformer model
        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

        # Log
        pbar.update()
        pbar.set_postfix(
            loss=f"{batch_loss:.2f}",
            accuracy=f"{batch_accuracy:.2f}",
            step=step + 1,
        )

        # Do validation
        if (step + 1) % valid_steps == 0:
            pbar.close()

            valid_accuracy = valid(valid_loader, transformer_model, criterion, device)

            # keep the best model
            if valid_accuracy > best_accuracy:
                best_accuracy = valid_accuracy
                best_state_dict = transformer_model.state_dict()

            pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

        # Save the best model so far.
        if (step + 1) % save_steps == 0 and best_state_dict is not None:
            torch.save(best_state_dict, save_path)
            pbar.write(f"Step {step + 1}, best model saved. (accuracy={best_accuracy:.4f})")

    pbar.close()


if __name__ == "__main__":
    main(**parse_args())

[Info]: Use cuda now!
[Info]: Finish loading data!
[Info]: Finish creating model!


Train: 100% 2000/2000 [00:55<00:00, 36.00 step/s, accuracy=0.16, loss=3.60, step=2000]
Valid: 100% 5664/5667 [00:04<00:00, 1265.40 uttr/s, accuracy=0.22, loss=3.83]
Train: 100% 2000/2000 [00:47<00:00, 42.38 step/s, accuracy=0.44, loss=3.08, step=4000]
Valid: 100% 5664/5667 [00:04<00:00, 1170.53 uttr/s, accuracy=0.38, loss=2.87]
Train: 100% 2000/2000 [00:50<00:00, 39.52 step/s, accuracy=0.50, loss=2.06, step=6000]
Valid: 100% 5664/5667 [00:03<00:00, 1800.93 uttr/s, accuracy=0.46, loss=2.45]
Train: 100% 2000/2000 [00:49<00:00, 40.70 step/s, accuracy=0.62, loss=1.56, step=8000]
Valid: 100% 5664/5667 [00:03<00:00, 1797.76 uttr/s, accuracy=0.51, loss=2.19]
Train: 100% 2000/2000 [00:51<00:00, 39.13 step/s, accuracy=0.44, loss=2.62, step=1e+4]
Valid: 100% 5664/5667 [00:03<00:00, 1515.03 uttr/s, accuracy=0.54, loss=2.05]
Train:   1% 11/2000 [00:00<00:33, 58.55 step/s, accuracy=0.56, loss=1.68, step=1e+4]

Step 10000, best model saved. (accuracy=0.5355)


Train: 100% 2000/2000 [00:45<00:00, 43.74 step/s, accuracy=0.47, loss=2.18, step=12000]
Valid: 100% 5664/5667 [00:03<00:00, 1796.23 uttr/s, accuracy=0.56, loss=1.95]
Train: 100% 2000/2000 [00:44<00:00, 45.19 step/s, accuracy=0.66, loss=1.91, step=14000]
Valid: 100% 5664/5667 [00:03<00:00, 1739.33 uttr/s, accuracy=0.58, loss=1.81]
Train: 100% 2000/2000 [00:42<00:00, 47.52 step/s, accuracy=0.72, loss=1.32, step=16000]
Valid: 100% 5664/5667 [00:03<00:00, 1737.40 uttr/s, accuracy=0.62, loss=1.67]
Train: 100% 2000/2000 [00:41<00:00, 48.51 step/s, accuracy=0.69, loss=1.55, step=18000]
Valid: 100% 5664/5667 [00:03<00:00, 1599.07 uttr/s, accuracy=0.62, loss=1.64]
Train: 100% 2000/2000 [00:41<00:00, 48.04 step/s, accuracy=0.59, loss=1.83, step=2e+4] 
Valid: 100% 5664/5667 [00:03<00:00, 1784.12 uttr/s, accuracy=0.63, loss=1.62]
Train:   0% 10/2000 [00:00<00:36, 54.32 step/s, accuracy=0.59, loss=1.65, step=2e+4]

Step 20000, best model saved. (accuracy=0.6287)


...

Step 80000, best model saved. (accuracy=0.7622)


Train: 100% 2000/2000 [00:40<00:00, 49.80 step/s, accuracy=0.81, loss=0.69, step=82000]
Valid: 100% 5664/5667 [00:03<00:00, 1747.79 uttr/s, accuracy=0.76, loss=1.07]
Train: 100% 2000/2000 [00:44<00:00, 44.61 step/s, accuracy=0.75, loss=1.06, step=84000]
Valid: 100% 5664/5667 [00:03<00:00, 1829.51 uttr/s, accuracy=0.76, loss=1.04]
Train: 100% 2000/2000 [00:44<00:00, 45.00 step/s, accuracy=0.78, loss=0.48, step=86000]
Valid: 100% 5664/5667 [00:03<00:00, 1831.96 uttr/s, accuracy=0.76, loss=1.06]
Train: 100% 2000/2000 [00:44<00:00, 45.12 step/s, accuracy=0.81, loss=0.58, step=88000]
Valid: 100% 5664/5667 [00:03<00:00, 1676.91 uttr/s, accuracy=0.77, loss=1.05]
Train: 100% 2000/2000 [00:47<00:00, 42.10 step/s, accuracy=0.81, loss=0.64, step=9e+4] 
Valid: 100% 5664/5667 [00:03<00:00, 1855.84 uttr/s, accuracy=0.76, loss=1.06]
Train:   0% 10/2000 [00:00<00:39, 50.39 step/s, accuracy=0.84, loss=0.79, step=9e+4]

Step 90000, best model saved. (accuracy=0.7662)


Train: 100% 2000/2000 [00:45<00:00, 44.25 step/s, accuracy=0.88, loss=0.49, step=92000]
Valid: 100% 5664/5667 [00:03<00:00, 1762.96 uttr/s, accuracy=0.77, loss=1.02]
Train: 100% 2000/2000 [00:44<00:00, 44.92 step/s, accuracy=0.78, loss=0.78, step=94000]
Valid: 100% 5664/5667 [00:05<00:00, 1013.00 uttr/s, accuracy=0.76, loss=1.04]
Train: 100% 2000/2000 [00:43<00:00, 46.01 step/s, accuracy=0.94, loss=0.35, step=96000]
Valid: 100% 5664/5667 [00:03<00:00, 1639.06 uttr/s, accuracy=0.77, loss=1.03]
Train: 100% 2000/2000 [00:48<00:00, 41.44 step/s, accuracy=0.91, loss=0.51, step=98000]
Valid: 100% 5664/5667 [00:03<00:00, 1822.00 uttr/s, accuracy=0.77, loss=1.04]
Train: 100% 2000/2000 [00:47<00:00, 41.81 step/s, accuracy=0.84, loss=0.46, step=1e+5] 
Valid: 100% 5664/5667 [00:05<00:00, 1043.76 uttr/s, accuracy=0.77, loss=1.02]
Train:   0% 0/2000 [00:00<?, ? step/s]


Step 100000, best model saved. (accuracy=0.7724)


In [18]:
def parse_args():
    """arguments"""
    config = {
        "data_dir": "/kaggle/input/cs4650dataset/Dataset",
        "save_path": "model.ckpt",
        "batch_size": 32,
        "n_workers": 8,
        "valid_steps": 2000,
        "warmup_steps": 1000,
        "save_steps": 10000,
        "total_steps": 100000,
    }

    return config


def main(
    data_dir,
    save_path,
    batch_size,
    n_workers,
    valid_steps,
    warmup_steps,
    total_steps,
    save_steps,
):
    """Main function."""
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"[Info]: Use {device} now!")

    train_loader, valid_loader, speaker_num = get_dataloader(data_dir, batch_size, n_workers)
    train_iterator = iter(train_loader)
    print(f"[Info]: Finish loading data!",flush = True)

    transformer_model = TransformerClassifierPooling(n_spks=speaker_num).to(device)
    criterion = AMSoftmax()
    optimizer = AdamW(transformer_model.parameters(), lr=1e-3)
    scheduler = get_cosine_schedule_with_warmup(optimizer, warmup_steps, total_steps)
    print(f"[Info]: Finish creating model!",flush = True)

    best_accuracy = -1.0
    best_state_dict = None

    pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

    for step in range(total_steps):
        # Get data
        try:
            batch = next(train_iterator)
        except StopIteration:
            train_iterator = iter(train_loader)
            batch = next(train_iterator)

        loss, accuracy = model_fn(batch, transformer_model, criterion, device, True)
        batch_loss = loss.item()
        batch_accuracy = accuracy.item()

        # Updata transformer model
        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

        # Log
        pbar.update()
        pbar.set_postfix(
            loss=f"{batch_loss:.2f}",
            accuracy=f"{batch_accuracy:.2f}",
            step=step + 1,
        )

        # Do validation
        if (step + 1) % valid_steps == 0:
            pbar.close()

            valid_accuracy = valid(valid_loader, transformer_model, criterion, device)

            # keep the best model
            if valid_accuracy > best_accuracy:
                best_accuracy = valid_accuracy
                best_state_dict = transformer_model.state_dict()

            pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

        # Save the best model so far.
        if (step + 1) % save_steps == 0 and best_state_dict is not None:
            torch.save(best_state_dict, save_path)
            pbar.write(f"Step {step + 1}, best model saved. (accuracy={best_accuracy:.4f})")

    pbar.close()


if __name__ == "__main__":
    main(**parse_args())

[Info]: Use cuda now!
[Info]: Finish loading data!
[Info]: Finish creating model!


Train:   0% 0/2000 [00:00<?, ? step/s]/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:16: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at /usr/local/src/pytorch/aten/src/ATen/native/IndexingUtils.h:27.)
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
Train: 100% 2000/2000 [00:48<00:00, 41.65 step/s, accuracy=0.06, loss=17.42, step=2000]
Valid: 100% 5664/5667 [00:03<00:00, 1712.28 uttr/s, accuracy=0.06, loss=8.88]
Train: 100% 2000/2000 [01:18<00:00, 25.58 step/s, accuracy=0.28, loss=15.26, step=4000]
Valid: 100% 5664/5667 [00:18<00:00, 308.01 uttr/s, accuracy=0.27, loss=7.70]
Train: 100% 2000/2000 [01:10<00:00, 28.18 step/s, accuracy=0.53, loss=12.58, step=6000]
Valid: 100% 5664/5667 [00:03<00:00, 1699.15 uttr/s, accuracy=0.38, loss=6.71]
Train: 100% 20

Step 10000, best model saved. (accuracy=0.5175)


Train: 100% 2000/2000 [00:52<00:00, 38.28 step/s, accuracy=0.50, loss=11.99, step=12000]
Valid: 100% 5664/5667 [00:03<00:00, 1523.57 uttr/s, accuracy=0.56, loss=4.66]
Train: 100% 2000/2000 [00:48<00:00, 40.97 step/s, accuracy=0.62, loss=11.83, step=14000]
Valid: 100% 5664/5667 [00:03<00:00, 1780.10 uttr/s, accuracy=0.58, loss=4.57]
Train: 100% 2000/2000 [00:52<00:00, 38.06 step/s, accuracy=0.78, loss=8.38, step=16000] 
Valid: 100% 5664/5667 [00:07<00:00, 716.67 uttr/s, accuracy=0.59, loss=4.27] 
Train: 100% 2000/2000 [00:50<00:00, 39.82 step/s, accuracy=0.62, loss=9.23, step=18000] 
Valid: 100% 5664/5667 [00:03<00:00, 1768.39 uttr/s, accuracy=0.61, loss=4.04]
Train: 100% 2000/2000 [00:47<00:00, 42.04 step/s, accuracy=0.81, loss=5.88, step=2e+4]  
Valid: 100% 5664/5667 [00:04<00:00, 1164.51 uttr/s, accuracy=0.62, loss=3.93]
Train:   0% 10/2000 [00:00<00:38, 51.35 step/s, accuracy=0.66, loss=8.28, step=2e+4]

Step 20000, best model saved. (accuracy=0.6194)


...

Step 80000, best model saved. (accuracy=0.7678)


Train: 100% 2000/2000 [00:46<00:00, 43.10 step/s, accuracy=0.72, loss=8.73, step=82000] 
Valid: 100% 5664/5667 [00:03<00:00, 1772.73 uttr/s, accuracy=0.76, loss=2.50]
Train: 100% 2000/2000 [00:50<00:00, 39.46 step/s, accuracy=0.81, loss=6.15, step=84000] 
Valid: 100% 5664/5667 [00:03<00:00, 1772.19 uttr/s, accuracy=0.77, loss=2.51]
Train: 100% 2000/2000 [00:48<00:00, 41.44 step/s, accuracy=0.84, loss=4.40, step=86000] 
Valid: 100% 5664/5667 [00:04<00:00, 1141.27 uttr/s, accuracy=0.77, loss=2.47]
Train: 100% 2000/2000 [00:47<00:00, 41.93 step/s, accuracy=0.81, loss=5.58, step=88000] 
Valid: 100% 5664/5667 [00:03<00:00, 1731.20 uttr/s, accuracy=0.78, loss=2.39]
Train: 100% 2000/2000 [00:48<00:00, 40.93 step/s, accuracy=0.81, loss=5.51, step=9e+4]  
Valid: 100% 5664/5667 [00:03<00:00, 1810.04 uttr/s, accuracy=0.77, loss=2.42]
Train:   0% 10/2000 [00:00<00:39, 50.24 step/s, accuracy=0.78, loss=5.78, step=9e+4]

Step 90000, best model saved. (accuracy=0.7809)


Train: 100% 2000/2000 [00:46<00:00, 43.09 step/s, accuracy=0.84, loss=4.15, step=92000] 
Valid: 100% 5664/5667 [00:04<00:00, 1345.88 uttr/s, accuracy=0.78, loss=2.39]
Train: 100% 2000/2000 [00:47<00:00, 42.53 step/s, accuracy=0.78, loss=6.04, step=94000]
Valid: 100% 5664/5667 [00:03<00:00, 1765.27 uttr/s, accuracy=0.77, loss=2.45]
Train: 100% 2000/2000 [00:45<00:00, 43.70 step/s, accuracy=0.94, loss=2.81, step=96000]
Valid: 100% 5664/5667 [00:04<00:00, 1304.30 uttr/s, accuracy=0.77, loss=2.44]
Train: 100% 2000/2000 [00:47<00:00, 42.37 step/s, accuracy=0.84, loss=4.85, step=98000] 
Valid: 100% 5664/5667 [00:03<00:00, 1551.03 uttr/s, accuracy=0.78, loss=2.42]
Train: 100% 2000/2000 [00:46<00:00, 42.87 step/s, accuracy=0.84, loss=4.21, step=1e+5]  
Valid: 100% 5664/5667 [00:03<00:00, 1752.07 uttr/s, accuracy=0.78, loss=2.40]
Train:   0% 0/2000 [00:00<?, ? step/s]


Step 100000, best model saved. (accuracy=0.7809)


# Conformer Model Train

In [15]:
def parse_args():
    """arguments"""
    config = {
        "data_dir": "../input/cs4650dataset/Dataset",
        "save_path": "model.ckpt",
        "batch_size": 32,
        "n_workers": 8,
        "valid_steps": 2000,
        "warmup_steps": 1000,
        "save_steps": 10000,
        "total_steps": 100000,
    }

    return config


def main(
    data_dir,
    save_path,
    batch_size,
    n_workers,
    valid_steps,
    warmup_steps,
    total_steps,
    save_steps,
):
    """Main function."""
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"[Info]: Use {device} now!")

    train_loader, valid_loader, speaker_num = get_dataloader(data_dir, batch_size, n_workers)
    train_iterator = iter(train_loader)
    print(f"[Info]: Finish loading data!",flush = True)

    conformer_model = ConformerClassifier(n_spks=speaker_num).to(device)
    criterion = CrossEntropyLoss()
    optimizer = AdamW(conformer_model.parameters(), lr=1e-3)
    scheduler = get_cosine_schedule_with_warmup(optimizer, warmup_steps, total_steps)
    print(f"[Info]: Finish creating model!",flush = True)

    best_accuracy = -1.0
    best_state_dict = None

    pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

    for step in range(total_steps):
        # Get data
        try:
            batch = next(train_iterator)
        except StopIteration:
            train_iterator = iter(train_loader)
            batch = next(train_iterator)

        loss, accuracy = model_fn(batch, conformer_model, criterion, device)
        batch_loss = loss.item()
        batch_accuracy = accuracy.item()

        # Updata transformer model
        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

        # Log
        pbar.update()
        pbar.set_postfix(
            loss=f"{batch_loss:.2f}",
            accuracy=f"{batch_accuracy:.2f}",
            step=step + 1,
        )

        # Do validation
        if (step + 1) % valid_steps == 0:
            pbar.close()

            valid_accuracy = valid(valid_loader, conformer_model, criterion, device)

            # keep the best model
            if valid_accuracy > best_accuracy:
                best_accuracy = valid_accuracy
                best_state_dict = conformer_model.state_dict()

            pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

        # Save the best model so far.
        if (step + 1) % save_steps == 0 and best_state_dict is not None:
            torch.save(best_state_dict, save_path)
            pbar.write(f"Step {step + 1}, best model saved. (accuracy={best_accuracy:.4f})")

    pbar.close()


if __name__ == "__main__":
    main(**parse_args())

[Info]: Use cuda now!


/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


[Info]: Finish loading data!
[Info]: Finish creating model!


Train: 100% 2000/2000 [01:54<00:00, 17.47 step/s, accuracy=0.06, loss=4.24, step=2000]
Valid: 100% 5664/5667 [00:06<00:00, 838.89 uttr/s, accuracy=0.14, loss=4.15] 
Train: 100% 2000/2000 [01:34<00:00, 21.09 step/s, accuracy=0.31, loss=3.27, step=4000]
Valid: 100% 5664/5667 [00:04<00:00, 1345.04 uttr/s, accuracy=0.24, loss=3.39]
Train: 100% 2000/2000 [01:34<00:00, 21.22 step/s, accuracy=0.41, loss=2.49, step=6000]
Valid: 100% 5664/5667 [00:04<00:00, 1344.76 uttr/s, accuracy=0.31, loss=3.01]
Train: 100% 2000/2000 [01:34<00:00, 21.10 step/s, accuracy=0.47, loss=2.70, step=8000]
Valid: 100% 5664/5667 [00:04<00:00, 1182.18 uttr/s, accuracy=0.38, loss=2.68]
Train: 100% 2000/2000 [01:37<00:00, 20.60 step/s, accuracy=0.38, loss=2.40, step=1e+4]
Valid: 100% 5664/5667 [00:04<00:00, 1357.64 uttr/s, accuracy=0.39, loss=2.63]
Train:   0% 4/2000 [00:00<01:53, 17.65 step/s, accuracy=0.19, loss=3.03, step=1e+4]

Step 10000, best model saved. (accuracy=0.3928)


...

Step 80000, best model saved. (accuracy=0.7873)


Train: 100% 2000/2000 [01:38<00:00, 20.23 step/s, accuracy=0.97, loss=0.38, step=82000]
Valid: 100% 5664/5667 [00:05<00:00, 1016.04 uttr/s, accuracy=0.78, loss=0.92]
Train: 100% 2000/2000 [01:40<00:00, 19.84 step/s, accuracy=0.97, loss=0.16, step=84000]
Valid: 100% 5664/5667 [00:04<00:00, 1307.80 uttr/s, accuracy=0.78, loss=0.91]
Train: 100% 2000/2000 [01:38<00:00, 20.37 step/s, accuracy=0.88, loss=0.59, step=86000]
Valid: 100% 5664/5667 [00:04<00:00, 1293.04 uttr/s, accuracy=0.79, loss=0.92]
Train: 100% 2000/2000 [01:39<00:00, 20.14 step/s, accuracy=0.88, loss=0.43, step=88000]
Valid: 100% 5664/5667 [00:04<00:00, 1352.06 uttr/s, accuracy=0.80, loss=0.89]
Train: 100% 2000/2000 [01:38<00:00, 20.21 step/s, accuracy=0.81, loss=0.51, step=9e+4] 
Valid: 100% 5664/5667 [00:04<00:00, 1312.68 uttr/s, accuracy=0.79, loss=0.93]
Train:   0% 3/2000 [00:00<02:02, 16.30 step/s, accuracy=0.91, loss=0.39, step=9e+4]

Step 90000, best model saved. (accuracy=0.7994)


Train: 100% 2000/2000 [01:39<00:00, 20.01 step/s, accuracy=1.00, loss=0.19, step=92000]
Valid: 100% 5664/5667 [00:04<00:00, 1224.35 uttr/s, accuracy=0.80, loss=0.87]
Train: 100% 2000/2000 [01:38<00:00, 20.36 step/s, accuracy=0.97, loss=0.15, step=94000]
Valid: 100% 5664/5667 [00:04<00:00, 1334.09 uttr/s, accuracy=0.80, loss=0.87]
Train: 100% 2000/2000 [01:37<00:00, 20.41 step/s, accuracy=0.84, loss=0.58, step=96000]
Valid: 100% 5664/5667 [00:04<00:00, 1336.38 uttr/s, accuracy=0.80, loss=0.89]
Train: 100% 2000/2000 [01:37<00:00, 20.51 step/s, accuracy=0.94, loss=0.23, step=98000]
Valid: 100% 5664/5667 [00:04<00:00, 1301.96 uttr/s, accuracy=0.79, loss=0.87]
Train: 100% 2000/2000 [01:37<00:00, 20.56 step/s, accuracy=0.81, loss=0.60, step=1e+5] 
Valid: 100% 5664/5667 [00:04<00:00, 1358.10 uttr/s, accuracy=0.79, loss=0.89]
Train:   0% 0/2000 [00:00<?, ? step/s]


Step 100000, best model saved. (accuracy=0.8003)


In [16]:
def parse_args():
    """arguments"""
    config = {
        "data_dir": "../input/cs4650dataset/Dataset",
        "save_path": "model.ckpt",
        "batch_size": 32,
        "n_workers": 8,
        "valid_steps": 2000,
        "warmup_steps": 1000,
        "save_steps": 10000,
        "total_steps": 100000,
    }

    return config


def main(
    data_dir,
    save_path,
    batch_size,
    n_workers,
    valid_steps,
    warmup_steps,
    total_steps,
    save_steps,
):
    """Main function."""
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"[Info]: Use {device} now!")

    train_loader, valid_loader, speaker_num = get_dataloader(data_dir, batch_size, n_workers)
    train_iterator = iter(train_loader)
    print(f"[Info]: Finish loading data!",flush = True)

    conformer_model = ConformerClassifier(n_spks=speaker_num).to(device)
    criterion = AMSoftmax()
    optimizer = AdamW(conformer_model.parameters(), lr=1e-3)
    scheduler = get_cosine_schedule_with_warmup(optimizer, warmup_steps, total_steps)
    print(f"[Info]: Finish creating model!",flush = True)

    best_accuracy = -1.0
    best_state_dict = None

    pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

    for step in range(total_steps):
        # Get data
        try:
            batch = next(train_iterator)
        except StopIteration:
            train_iterator = iter(train_loader)
            batch = next(train_iterator)

        loss, accuracy = model_fn(batch, conformer_model, criterion, device, True)
        batch_loss = loss.item()
        batch_accuracy = accuracy.item()

        # Updata transformer model
        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

        # Log
        pbar.update()
        pbar.set_postfix(
            loss=f"{batch_loss:.2f}",
            accuracy=f"{batch_accuracy:.2f}",
            step=step + 1,
        )

        # Do validation
        if (step + 1) % valid_steps == 0:
            pbar.close()

            valid_accuracy = valid(valid_loader, conformer_model, criterion, device)

            # keep the best model
            if valid_accuracy > best_accuracy:
                best_accuracy = valid_accuracy
                best_state_dict = conformer_model.state_dict()

            pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

        # Save the best model so far.
        if (step + 1) % save_steps == 0 and best_state_dict is not None:
            torch.save(best_state_dict, save_path)
            pbar.write(f"Step {step + 1}, best model saved. (accuracy={best_accuracy:.4f})")

    pbar.close()


if __name__ == "__main__":
    main(**parse_args())

[Info]: Use cuda now!
[Info]: Finish loading data!
[Info]: Finish creating model!


Train:   0% 0/2000 [00:00<?, ? step/s]/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:16: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at /usr/local/src/pytorch/aten/src/ATen/native/IndexingUtils.h:27.)
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
/opt/conda/lib/python3.7/site-packages/torch/autograd/__init__.py:199: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at /usr/local/src/pytorch/aten/src/ATen/native/IndexingUtils.h:27.)
  allow_unreachable=True, accumulate_grad=True)  # Calls into the C++ engine to run the backward pass
Train: 100% 2000/2000 [01:39<00:00, 20.05 step/s, accuracy=0.03, loss=17.23, step=2000]
Valid: 100% 5664/5667 [00:04<00:00, 1196.39 uttr/s, accur

Step 10000, best model saved. (accuracy=0.0750)


Train: 100% 2000/2000 [01:39<00:00, 20.05 step/s, accuracy=0.06, loss=16.17, step=12000]
Valid: 100% 5664/5667 [00:04<00:00, 1275.45 uttr/s, accuracy=0.07, loss=8.41]
Train: 100% 2000/2000 [01:39<00:00, 20.06 step/s, accuracy=0.06, loss=15.59, step=14000]
Valid: 100% 5664/5667 [00:04<00:00, 1334.72 uttr/s, accuracy=0.09, loss=8.33]
Train: 100% 2000/2000 [01:39<00:00, 20.12 step/s, accuracy=0.06, loss=16.46, step=16000]
Valid: 100% 5664/5667 [00:04<00:00, 1344.57 uttr/s, accuracy=0.09, loss=8.28]
Train: 100% 2000/2000 [01:40<00:00, 19.98 step/s, accuracy=0.12, loss=16.54, step=18000]
Valid: 100% 5664/5667 [00:04<00:00, 1327.28 uttr/s, accuracy=0.09, loss=8.15]
Train: 100% 2000/2000 [01:40<00:00, 19.87 step/s, accuracy=0.12, loss=16.22, step=2e+4] 
Valid: 100% 5664/5667 [00:04<00:00, 1289.06 uttr/s, accuracy=0.09, loss=8.17]
Train:   0% 3/2000 [00:00<02:00, 16.64 step/s, accuracy=0.09, loss=16.01, step=2e+4]

Step 20000, best model saved. (accuracy=0.0927)


...

Step 80000, best model saved. (accuracy=0.6623)


Train: 100% 2000/2000 [01:39<00:00, 20.15 step/s, accuracy=0.72, loss=6.95, step=82000] 
Valid: 100% 5664/5667 [00:04<00:00, 1313.10 uttr/s, accuracy=0.67, loss=3.42]
Train: 100% 2000/2000 [01:37<00:00, 20.58 step/s, accuracy=0.69, loss=7.36, step=84000] 
Valid: 100% 5664/5667 [00:04<00:00, 1289.17 uttr/s, accuracy=0.67, loss=3.41]
Train: 100% 2000/2000 [01:37<00:00, 20.46 step/s, accuracy=0.91, loss=5.34, step=86000] 
Valid: 100% 5664/5667 [00:04<00:00, 1354.98 uttr/s, accuracy=0.68, loss=3.33]
Train: 100% 2000/2000 [01:39<00:00, 20.15 step/s, accuracy=0.81, loss=5.69, step=88000] 
Valid: 100% 5664/5667 [00:04<00:00, 1262.61 uttr/s, accuracy=0.67, loss=3.33]
Train: 100% 2000/2000 [01:37<00:00, 20.45 step/s, accuracy=0.69, loss=7.12, step=9e+4]  
Valid: 100% 5664/5667 [00:04<00:00, 1182.28 uttr/s, accuracy=0.68, loss=3.30]
Train:   0% 4/2000 [00:00<01:55, 17.35 step/s, accuracy=0.88, loss=3.36, step=9e+4]

Step 90000, best model saved. (accuracy=0.6847)


Train: 100% 2000/2000 [01:38<00:00, 20.29 step/s, accuracy=0.62, loss=9.20, step=92000] 
Valid: 100% 5664/5667 [00:04<00:00, 1321.86 uttr/s, accuracy=0.69, loss=3.26]
Train: 100% 2000/2000 [01:38<00:00, 20.38 step/s, accuracy=0.88, loss=4.88, step=94000] 
Valid: 100% 5664/5667 [00:04<00:00, 1262.59 uttr/s, accuracy=0.69, loss=3.21]
Train: 100% 2000/2000 [01:37<00:00, 20.48 step/s, accuracy=0.78, loss=5.61, step=96000] 
Valid: 100% 5664/5667 [00:04<00:00, 1270.39 uttr/s, accuracy=0.69, loss=3.24]
Train: 100% 2000/2000 [01:41<00:00, 19.72 step/s, accuracy=0.69, loss=7.96, step=98000] 
Valid: 100% 5664/5667 [00:04<00:00, 1143.48 uttr/s, accuracy=0.68, loss=3.29]
Train: 100% 2000/2000 [01:41<00:00, 19.67 step/s, accuracy=0.72, loss=7.37, step=1e+5]  
Valid: 100% 5664/5667 [00:04<00:00, 1307.01 uttr/s, accuracy=0.69, loss=3.27]
Train:   0% 0/2000 [00:00<?, ? step/s]


Step 100000, best model saved. (accuracy=0.6930)


In [18]:
def parse_args():
    """arguments"""
    config = {
        "data_dir": "../input/cs4650dataset/Dataset",
        "save_path": "model.ckpt",
        "batch_size": 32,
        "n_workers": 8,
        "valid_steps": 2000,
        "warmup_steps": 1000,
        "save_steps": 10000,
        "total_steps": 100000,
    }

    return config


def main(
    data_dir,
    save_path,
    batch_size,
    n_workers,
    valid_steps,
    warmup_steps,
    total_steps,
    save_steps,
):
    """Main function."""
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"[Info]: Use {device} now!")

    train_loader, valid_loader, speaker_num = get_dataloader(data_dir, batch_size, n_workers)
    train_iterator = iter(train_loader)
    print(f"[Info]: Finish loading data!",flush = True)

    conformer_model = ConformerClassifierPooling(n_spks=speaker_num).to(device)
    criterion = CrossEntropyLoss()
    optimizer = AdamW(conformer_model.parameters(), lr=1e-3)
    scheduler = get_cosine_schedule_with_warmup(optimizer, warmup_steps, total_steps)
    print(f"[Info]: Finish creating model!",flush = True)

    best_accuracy = -1.0
    best_state_dict = None

    pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

    for step in range(total_steps):
        # Get data
        try:
            batch = next(train_iterator)
        except StopIteration:
            train_iterator = iter(train_loader)
            batch = next(train_iterator)

        loss, accuracy = model_fn(batch, conformer_model, criterion, device)
        batch_loss = loss.item()
        batch_accuracy = accuracy.item()

        # Updata transformer model
        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

        # Log
        pbar.update()
        pbar.set_postfix(
            loss=f"{batch_loss:.2f}",
            accuracy=f"{batch_accuracy:.2f}",
            step=step + 1,
        )

        # Do validation
        if (step + 1) % valid_steps == 0:
            pbar.close()

            valid_accuracy = valid(valid_loader, conformer_model, criterion, device)

            # keep the best model
            if valid_accuracy > best_accuracy:
                best_accuracy = valid_accuracy
                best_state_dict = conformer_model.state_dict()

            pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

        # Save the best model so far.
        if (step + 1) % save_steps == 0 and best_state_dict is not None:
            torch.save(best_state_dict, save_path)
            pbar.write(f"Step {step + 1}, best model saved. (accuracy={best_accuracy:.4f})")

    pbar.close()


if __name__ == "__main__":
    main(**parse_args())

[Info]: Use cuda now!
[Info]: Finish loading data!
[Info]: Finish creating model!


Train: 100% 2000/2000 [01:37<00:00, 20.48 step/s, accuracy=0.16, loss=4.08, step=2000]
Valid: 100% 5664/5667 [00:04<00:00, 1166.03 uttr/s, accuracy=0.20, loss=3.79]
Train: 100% 2000/2000 [01:37<00:00, 20.54 step/s, accuracy=0.34, loss=3.09, step=4000]
Valid: 100% 5664/5667 [00:04<00:00, 1345.94 uttr/s, accuracy=0.38, loss=2.75]
Train: 100% 2000/2000 [01:36<00:00, 20.68 step/s, accuracy=0.53, loss=2.06, step=6000]
Valid: 100% 5664/5667 [00:04<00:00, 1345.32 uttr/s, accuracy=0.49, loss=2.23]
Train: 100% 2000/2000 [01:37<00:00, 20.54 step/s, accuracy=0.50, loss=2.11, step=8000]
Valid: 100% 5664/5667 [00:04<00:00, 1325.12 uttr/s, accuracy=0.55, loss=1.96]
Train: 100% 2000/2000 [01:38<00:00, 20.41 step/s, accuracy=0.72, loss=1.27, step=1e+4]
Valid: 100% 5664/5667 [00:04<00:00, 1307.04 uttr/s, accuracy=0.61, loss=1.63]
Train:   0% 4/2000 [00:00<01:50, 18.09 step/s, accuracy=0.69, loss=1.19, step=1e+4]

Step 10000, best model saved. (accuracy=0.6130)


Train: 100% 2000/2000 [01:37<00:00, 20.53 step/s, accuracy=0.69, loss=1.59, step=12000]
Valid: 100% 5664/5667 [00:04<00:00, 1169.83 uttr/s, accuracy=0.63, loss=1.54]
Train: 100% 2000/2000 [01:37<00:00, 20.52 step/s, accuracy=0.59, loss=1.60, step=14000]
Valid: 100% 5664/5667 [00:04<00:00, 1365.93 uttr/s, accuracy=0.67, loss=1.36]
Train: 100% 2000/2000 [01:37<00:00, 20.42 step/s, accuracy=0.75, loss=1.43, step=16000]
Valid: 100% 5664/5667 [00:04<00:00, 1306.32 uttr/s, accuracy=0.69, loss=1.28]
Train: 100% 2000/2000 [01:37<00:00, 20.53 step/s, accuracy=0.69, loss=1.15, step=18000]
Valid: 100% 5664/5667 [00:04<00:00, 1309.89 uttr/s, accuracy=0.73, loss=1.15]
Train: 100% 2000/2000 [01:37<00:00, 20.42 step/s, accuracy=0.88, loss=0.46, step=2e+4] 
Valid: 100% 5664/5667 [00:04<00:00, 1328.25 uttr/s, accuracy=0.72, loss=1.16]
Train:   0% 4/2000 [00:00<01:54, 17.42 step/s, accuracy=0.81, loss=0.58, step=2e+4]

Step 20000, best model saved. (accuracy=0.7256)


...

Step 80000, best model saved. (accuracy=0.8861)


Train: 100% 2000/2000 [01:41<00:00, 19.77 step/s, accuracy=1.00, loss=0.05, step=82000]
Valid: 100% 5664/5667 [00:04<00:00, 1290.16 uttr/s, accuracy=0.88, loss=0.54]
Train: 100% 2000/2000 [01:42<00:00, 19.47 step/s, accuracy=0.97, loss=0.07, step=84000]
Valid: 100% 5664/5667 [00:04<00:00, 1317.46 uttr/s, accuracy=0.88, loss=0.54]
Train: 100% 2000/2000 [01:40<00:00, 19.95 step/s, accuracy=1.00, loss=0.02, step=86000]
Valid: 100% 5664/5667 [00:04<00:00, 1341.80 uttr/s, accuracy=0.88, loss=0.52]
Train: 100% 2000/2000 [01:41<00:00, 19.66 step/s, accuracy=1.00, loss=0.03, step=88000]
Valid: 100% 5664/5667 [00:04<00:00, 1280.98 uttr/s, accuracy=0.88, loss=0.52]
Train: 100% 2000/2000 [01:40<00:00, 19.84 step/s, accuracy=1.00, loss=0.02, step=9e+4] 
Valid: 100% 5664/5667 [00:04<00:00, 1300.39 uttr/s, accuracy=0.89, loss=0.51]
Train:   0% 3/2000 [00:00<01:56, 17.13 step/s, accuracy=0.94, loss=0.20, step=9e+4]

Step 90000, best model saved. (accuracy=0.8909)


Train: 100% 2000/2000 [01:41<00:00, 19.80 step/s, accuracy=0.94, loss=0.16, step=92000]
Valid: 100% 5664/5667 [00:04<00:00, 1303.69 uttr/s, accuracy=0.89, loss=0.53]
Train: 100% 2000/2000 [01:40<00:00, 19.85 step/s, accuracy=1.00, loss=0.02, step=94000]
Valid: 100% 5664/5667 [00:04<00:00, 1178.59 uttr/s, accuracy=0.89, loss=0.49]
Train: 100% 2000/2000 [01:39<00:00, 20.15 step/s, accuracy=0.97, loss=0.30, step=96000]
Valid: 100% 5664/5667 [00:04<00:00, 1300.72 uttr/s, accuracy=0.89, loss=0.49]
Train: 100% 2000/2000 [01:38<00:00, 20.22 step/s, accuracy=1.00, loss=0.02, step=98000]
Valid: 100% 5664/5667 [00:04<00:00, 1331.62 uttr/s, accuracy=0.89, loss=0.52]
Train: 100% 2000/2000 [01:38<00:00, 20.30 step/s, accuracy=1.00, loss=0.06, step=1e+5] 
Valid: 100% 5664/5667 [00:04<00:00, 1288.19 uttr/s, accuracy=0.89, loss=0.50]
Train:   0% 0/2000 [00:00<?, ? step/s]


Step 100000, best model saved. (accuracy=0.8928)


In [17]:
def parse_args():
    """arguments"""
    config = {
        "data_dir": "../input/cs4650dataset/Dataset",
        "save_path": "model.ckpt",
        "batch_size": 32,
        "n_workers": 8,
        "valid_steps": 2000,
        "warmup_steps": 1000,
        "save_steps": 10000,
        "total_steps": 100000,
    }

    return config


def main(
    data_dir,
    save_path,
    batch_size,
    n_workers,
    valid_steps,
    warmup_steps,
    total_steps,
    save_steps,
):
    """Main function."""
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"[Info]: Use {device} now!")

    train_loader, valid_loader, speaker_num = get_dataloader(data_dir, batch_size, n_workers)
    train_iterator = iter(train_loader)
    print(f"[Info]: Finish loading data!",flush = True)

    conformer_model = ConformerClassifierPooling(n_spks=speaker_num).to(device)
    criterion = AMSoftmax()
    optimizer = AdamW(conformer_model.parameters(), lr=1e-3)
    scheduler = get_cosine_schedule_with_warmup(optimizer, warmup_steps, total_steps)
    print(f"[Info]: Finish creating model!",flush = True)

    best_accuracy = -1.0
    best_state_dict = None

    pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

    for step in range(total_steps):
        # Get data
        try:
            batch = next(train_iterator)
        except StopIteration:
            train_iterator = iter(train_loader)
            batch = next(train_iterator)

        loss, accuracy = model_fn(batch, conformer_model, criterion, device, True)
        batch_loss = loss.item()
        batch_accuracy = accuracy.item()

        # Updata transformer model
        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

        # Log
        pbar.update()
        pbar.set_postfix(
            loss=f"{batch_loss:.2f}",
            accuracy=f"{batch_accuracy:.2f}",
            step=step + 1,
        )

        # Do validation
        if (step + 1) % valid_steps == 0:
            pbar.close()

            valid_accuracy = valid(valid_loader, conformer_model, criterion, device)

            # keep the best model
            if valid_accuracy > best_accuracy:
                best_accuracy = valid_accuracy
                best_state_dict = conformer_model.state_dict()

            pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

        # Save the best model so far.
        if (step + 1) % save_steps == 0 and best_state_dict is not None:
            torch.save(best_state_dict, save_path)
            pbar.write(f"Step {step + 1}, best model saved. (accuracy={best_accuracy:.4f})")

    pbar.close()


if __name__ == "__main__":
    main(**parse_args())

[Info]: Use cuda now!
[Info]: Finish loading data!
[Info]: Finish creating model!


Train:   0% 0/2000 [00:00<?, ? step/s]/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:16: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at /usr/local/src/pytorch/aten/src/ATen/native/IndexingUtils.h:27.)
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
Train: 100% 2000/2000 [01:42<00:00, 19.50 step/s, accuracy=0.25, loss=15.80, step=2000]
Valid: 100% 5664/5667 [00:05<00:00, 1130.81 uttr/s, accuracy=0.17, loss=8.35]
Train: 100% 2000/2000 [01:40<00:00, 19.92 step/s, accuracy=0.28, loss=13.81, step=4000]
Valid: 100% 5664/5667 [00:04<00:00, 1312.12 uttr/s, accuracy=0.33, loss=7.08]
Train: 100% 2000/2000 [01:39<00:00, 20.07 step/s, accuracy=0.38, loss=13.91, step=6000]
Valid: 100% 5664/5667 [00:04<00:00, 1234.41 uttr/s, accuracy=0.43, loss=6.17]
Train: 100% 2

Step 10000, best model saved. (accuracy=0.6132)


Train: 100% 2000/2000 [01:41<00:00, 19.73 step/s, accuracy=0.66, loss=10.11, step=12000]
Valid: 100% 5664/5667 [00:04<00:00, 1296.14 uttr/s, accuracy=0.64, loss=3.64]
Train: 100% 2000/2000 [01:41<00:00, 19.68 step/s, accuracy=0.81, loss=6.21, step=14000] 
Valid: 100% 5664/5667 [00:04<00:00, 1230.42 uttr/s, accuracy=0.67, loss=3.41]
Train: 100% 2000/2000 [01:42<00:00, 19.49 step/s, accuracy=0.72, loss=7.67, step=16000] 
Valid: 100% 5664/5667 [00:04<00:00, 1293.30 uttr/s, accuracy=0.71, loss=2.96]
Train: 100% 2000/2000 [01:40<00:00, 19.93 step/s, accuracy=0.69, loss=9.02, step=18000] 
Valid: 100% 5664/5667 [00:04<00:00, 1310.11 uttr/s, accuracy=0.72, loss=2.92]
Train: 100% 2000/2000 [01:38<00:00, 20.37 step/s, accuracy=0.66, loss=9.35, step=2e+4]  
Valid: 100% 5664/5667 [00:04<00:00, 1288.20 uttr/s, accuracy=0.74, loss=2.69]
Train:   0% 3/2000 [00:00<01:57, 17.06 step/s, accuracy=0.97, loss=3.49, step=2e+4]

Step 20000, best model saved. (accuracy=0.7442)


...

Step 80000, best model saved. (accuracy=0.8775)


Train: 100% 2000/2000 [01:39<00:00, 20.06 step/s, accuracy=1.00, loss=0.08, step=82000]
Valid: 100% 5664/5667 [00:05<00:00, 1004.01 uttr/s, accuracy=0.88, loss=1.33]
Train: 100% 2000/2000 [01:39<00:00, 20.01 step/s, accuracy=1.00, loss=0.48, step=84000]
Valid: 100% 5664/5667 [00:04<00:00, 1269.77 uttr/s, accuracy=0.88, loss=1.31]
Train: 100% 2000/2000 [01:38<00:00, 20.36 step/s, accuracy=0.97, loss=0.90, step=86000]
Valid: 100% 5664/5667 [00:04<00:00, 1218.64 uttr/s, accuracy=0.89, loss=1.25]
Train: 100% 2000/2000 [01:41<00:00, 19.75 step/s, accuracy=1.00, loss=0.83, step=88000]
Valid: 100% 5664/5667 [00:04<00:00, 1271.65 uttr/s, accuracy=0.89, loss=1.28]
Train: 100% 2000/2000 [01:39<00:00, 20.00 step/s, accuracy=0.97, loss=0.73, step=9e+4] 
Valid: 100% 5664/5667 [00:04<00:00, 1274.34 uttr/s, accuracy=0.88, loss=1.27]
Train:   0% 4/2000 [00:00<02:01, 16.48 step/s, accuracy=0.97, loss=0.79, step=9e+4]

Step 90000, best model saved. (accuracy=0.8900)


Train: 100% 2000/2000 [01:42<00:00, 19.54 step/s, accuracy=0.97, loss=0.83, step=92000]
Valid: 100% 5664/5667 [00:04<00:00, 1314.50 uttr/s, accuracy=0.88, loss=1.29]
Train: 100% 2000/2000 [01:40<00:00, 19.96 step/s, accuracy=0.97, loss=1.16, step=94000]
Valid: 100% 5664/5667 [00:04<00:00, 1266.44 uttr/s, accuracy=0.89, loss=1.26]
Train: 100% 2000/2000 [01:39<00:00, 20.19 step/s, accuracy=1.00, loss=0.01, step=96000]
Valid: 100% 5664/5667 [00:04<00:00, 1297.57 uttr/s, accuracy=0.89, loss=1.26]
Train: 100% 2000/2000 [01:38<00:00, 20.24 step/s, accuracy=0.91, loss=2.67, step=98000]
Valid: 100% 5664/5667 [00:04<00:00, 1303.43 uttr/s, accuracy=0.88, loss=1.29]
Train: 100% 2000/2000 [01:38<00:00, 20.24 step/s, accuracy=1.00, loss=0.37, step=1e+5] 
Valid: 100% 5664/5667 [00:04<00:00, 1260.82 uttr/s, accuracy=0.89, loss=1.22]
Train:   0% 0/2000 [00:00<?, ? step/s]


Step 100000, best model saved. (accuracy=0.8928)


## Dataset of inference

In [ ]:
import os
import json
import torch
from pathlib import Path
from torch.utils.data import Dataset


class InferenceDataset(Dataset):
    def __init__(self, data_dir):
        testdata_path = Path(data_dir) / "testdata.json"
        metadata = json.load(testdata_path.open())
        self.data_dir = data_dir
        self.data = metadata["utterances"]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        utterance = self.data[index]
        feat_path = utterance["feature_path"]
        mel = torch.load(os.path.join(self.data_dir, feat_path))

        return feat_path, mel


    def inference_collate_batch(batch):
        """Collate a batch of data."""
        feat_paths, mels = zip(*batch)

        return feat_paths, torch.stack(mels)

## Main funcrion of Inference

In [ ]:
import json
import csv
from pathlib import Path
from tqdm.notebook import tqdm

import torch
from torch.utils.data import DataLoader

def parse_args():
    """arguments"""
    config = {
        "data_dir": "../input/cs4650dataset/Dataset",
        "model_path": "./model.ckpt",
        "output_path": "./output.csv",
    }

    return config


def main(
    data_dir,
    model_path,
    output_path,
):
    """Main function."""
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"[Info]: Use {device} now!")

    mapping_path = Path(data_dir) / "mapping.json"
    mapping = json.load(mapping_path.open())

    dataset = InferenceDataset(data_dir)
    dataloader = DataLoader(
        dataset,
        batch_size=1,
        shuffle=False,
        drop_last=False,
        num_workers=8,
        collate_fn=InferenceDataset.inference_collate_batch,
    )
    print(f"[Info]: Finish loading data!",flush = True)

    speaker_num = len(mapping["id2speaker"])
    model = ConformerClassifier(n_spks=speaker_num).to(device)
    model.load_state_dict(torch.load(model_path))
    model.eval()
    print(f"[Info]: Finish creating model!",flush = True)

    results = [["Id", "Category"]]
    for feat_paths, mels in tqdm(dataloader):
        with torch.no_grad():
            mels = mels.to(device)
            outs = model(mels)
            preds = outs.argmax(1).cpu().numpy()
            for feat_path, pred in zip(feat_paths, preds):
                results.append([feat_path, mapping["id2speaker"][str(pred)]])

    with open(output_path, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(results)


if __name__ == "__main__":
    main(**parse_args())